# Restore ELK Snapshot on Measurement Node

## Import

In [2]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

MFLib version  1.0.0 


## Slice Info

In [3]:
# Change your slice name and node name
slice_name = "MyMonitoredSlice"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

In [4]:
mf = MFLib(slice_name)

Inititializing slice "MyMonitoredSlice" for MeasurementFramework.
Found meas node as meas-node at 2620:0:c80:1001:f816:3eff:fead:27bf
Bootstrap status not found. Will now start bootstrap process...
Generating MFUser Keys...
MFUser key generation Done.
Installing mfuser account...
Installing mfuser account done.
Cloning Measurement Framework Repository from github.com...
Cloning Measurement Framework Repository from github.com done.
Configuring Measurement Network...
Measurement Network setup complete.
Generating Ansible Inventory for Measurement Framework Deployment...
Ansible Inventory for Measurement Framework Deployment generated and saved.
Bootstrapping measurement node via bash...
Starting Bootstrap Process on Measure Node (bash script)...
Bootstrap Process on Measure Node (bash script) done.
Bootstrapping measurement node via ansible...
Starting Bootstrap Process on Measure Node (Ansible Playbook)...
Bootstrap Process on Measure Node (Ansible Playbook) done.
Bootstrap ansible scr

In [5]:
meas_node_name = mf.measurement_node_name
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")   
#print (meas_node)

-----------  ------------------------------------
Slice Name   MyMonitoredSlice
Slice ID     b91a0c5d-c0cf-4773-a9cb-630ab1cd8d58
Slice State  StableOK
Lease End    2023-07-15 19:20:31 +0000
-----------  ------------------------------------


## Restore elk data using existing snapshot

#### Upload your snapshot file

In [19]:
# Enter your elk snapshot tar file name (must be in data_transfer_service/snapshots directory)
snapshot_tar_name = "elk_backup_2.tar"


meas_node.upload_file(f"./snapshots/{snapshot_tar_name}", f'/tmp/{snapshot_tar_name}')
stdout, stderr = meas_node.execute(f'sudo mkdir /home/ubuntu/snapshots') 
stdout, stderr = meas_node.execute(f'sudo mv /tmp/{snapshot_tar_name} /home/ubuntu/snapshots/{snapshot_tar_name}') 

#### View snapshots ready to be imported into ELK

In [22]:
stdout, stderr = meas_node.execute('ls /home/ubuntu/snapshots') 

elk_backup_2.tar


#### Import a snapshot

In [23]:
# Uncomment below and change name to a different file from above if you'd like to
# snapshot_tar_name = "elk_backup_2.tar"

In [26]:
stdout, stderr = meas_node.execute('sudo tar -xvf /home/ubuntu/snapshots/elk_backup_2.tar -C /home/ubuntu/snapshots') 

_data/
_data/snap-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/meta-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/indices/
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/snap-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/snap-zLt1yM6tRmmMpnqxZMjj2A.dat
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/index-qFWoddmvTICVl6YnJYEgWw
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/meta-Dbibi4gBMQKlzNGxJMF-.dat
_data/indices/16fzAGeARxaEMOT80y3Xrw/
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/snap-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/index-NLZmJaDSSty1NyT-4Agqbg
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/snap-zLt1yM6tRmmMpnqxZMjj2A.dat
_data/indices/16fzAGeARxaEMOT80y3Xrw/meta-DLibi4gBMQKlzNGxJMF9.dat
_data/indices/qIeeCZxXS8uiq4TicG7NOA/
_data/indices/qIeeCZxXS8uiq4TicG7NOA/0/
_data/indices/qIeeCZxXS8uiq4TicG7NOA/0/index-1kxLtEf4Tn6YgjdTHx1NMg
_data/indices/qIeeCZxXS8uiq4TicG7NOA/0/snap-nx

#### Register the repository

In [30]:
stdout, stderr = meas_node.execute('curl -X PUT -H "Content-Type: application/json" -d \'{ "type": "fs", "settings": { "location": "/var/lib/docker/volumes/elk_data01/_data", "compress": true } }\' http://localhost:9200/_snapshot/imported_repository')

{"error":{"root_cause":[{"type":"repository_exception","reason":"[imported_repository] location [/var/lib/docker/volumes/elk_data01/_data] doesn't match any of the locations specified by path.repo because this setting is empty"}],"type":"repository_exception","reason":"[imported_repository] failed to create repository","caused_by":{"type":"repository_exception","reason":"[imported_repository] location [/var/lib/docker/volumes/elk_data01/_data] doesn't match any of the locations specified by path.repo because this setting is empty"}},"status":500}   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   658  100   552  100   106  55200  10600 --:--:-- --:--:-- --:--:-- 65800
 

#### View repository after registration

In [ ]:
stdout, stderr = node.execute('curl -X GET "http://localhost:9200/_cat/snapshots/imported_repository?v"')

#### Delete all indices and restore snapshot (replace snapshot name and change index list if you want to customize)

In [ ]:
stdout, stderr = node.execute('curl -X DELETE "http://localhost:9200/_all"')
stdout, stderr = node.execute('curl -X POST "http://localhost:9200/_snapshot/imported_repository/snapshot_test/_restore?pretty" -H "Content-Type: application/json" -d \'{ "indices": "*", "index_settings": { "index.number_of_replicas": 1 } }\'')

### Your data is now imported on the node.

### Check existing snapshot in the repository

In [13]:
repo_name="backup_repository"
cmd = f"curl -X GET 'http://localhost:9200?v'"
stdout, stderr= meas_node.execute(cmd)

{"error":{"root_cause":[{"type":"illegal_argument_exception","reason":"request [/] contains unrecognized parameter: [v]"}],"type":"illegal_argument_exception","reason":"request [/] contains unrecognized parameter: [v]"},"status":400}   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   233  100   233    0     0   113k      0 --:--:-- --:--:-- --:--:--  113k
 

### Check the snapshot

In [12]:
snapshot_name="snapshot_test"
cmd = f'curl -X GET "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?pretty"'
stdout, stderr= meas_node.execute(cmd)

{
  "error" : {
    "root_cause" : [
      {
        "type" : "repository_exception",
        "reason" : "[backup_repository] could not read repository data from index blob"
      }
    ],
    "type" : "repository_exception",
    "reason" : "[backup_repository] Could not read repository data because the contents of the repository do not match its expected state. This is likely the result of either concurrently modifying the contents of the repository by a process other than this cluster or an issue with the repository's underlying storage. The repository has been disabled to prevent corrupting its contents. To re-enable it and continue using it please remove the repository from the cluster and add it again to make the cluster recover the known state of the repository from its physical contents. The last cluster to write to this repository was [A63ZXCiURES-wcm6MvFiSA] at generation [1].",
    "caused_by" : {
      "type" : "repository_exception",
      "reason" : "[backup_repository] co

In [11]:
# EDIT THIS - choose a list of index files from your local storage to import
local_files = ["./snapshots/elk_backup_2023-06-02_18:23:33.tar"]

output = mf.upload_service_files("elk", local_files)

{'success': True, 'message': 'Service file elk_backup_2023-06-02_18:23:33.tar uploaded successfully.'}


### Restore the data using specific snapshot

In [ ]:
# Need to specify indice names to restore, find in above cell and change below
indice_str= f'"indices": "filebeat-7.13.2-2023.05.29-000001,metricbeat-7.13.2-2023.05.29-000001,packetbeat-7.13.2-2023.05.29-000001",'
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f'curl -X POST "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}/_restore?pretty" -H "Content-Type: application/json" -d \'{{{indice_str} "rename_pattern": "(.+)", "rename_replacement": "restored-$1", "include_aliases": false}}\''
#print (cmd)
stdout, stderr= meas_node.execute(cmd)

### Check the progress

In [ ]:
cmd = f'curl -XGET "http://localhost:9200/_cat/recovery?v"'
stdout, stderr= meas_node.execute(cmd)

## Once finished, check indice

In [ ]:
cmd= f'curl -XGET "http://localhost:9200/_cat/indices?pretty"'
stdout, stderr= meas_node.execute(cmd)

## Remove the index life cycle policy

In [ ]:
restored_filebeat_index="restored-filebeat-7.13.2-2023.05.29-000001"
restored_metricbeat_index="restored-metricbeat-7.13.2-2023.05.29-000001"
restored_packetbeat_index = "restored-packetbeat-7.13.2-2023.05.29-000001"
cmd1 = f'curl -X POST "http://localhost:9200/{restored_filebeat_index}/_ilm/remove?pretty"'
cmd2 = f'curl -X POST "http://localhost:9200/{restored_metricbeat_index}/_ilm/remove?pretty"'
cmd3 = f'curl -X POST "http://localhost:9200/{restored_packetbeat_index}/_ilm/remove?pretty"'
stdout, stderr= meas_node.execute(cmd1)
stdout, stderr= meas_node.execute(cmd2)
stdout, stderr= meas_node.execute(cmd3)

## Set up ssh tunnel to view the data

In [ ]:
port = "10050"
cmd=f'ssh -L {port}:localhost:5601 -F ssh_config -i slice_key ubuntu@{meas_node.get_management_ip()}'
print (f"Use [{cmd}] to set up ssh tunnel and then go to http://localhost:{port} to view the data ")

### You will see (restored) filebeat/metricbeat/packetbeat indice(6 total). The restored ones are static with no life cycle policies. The regular filebeat/metricbeat/packetbeat indice will be continuous written to by the beats.   